In [2]:
from google.generativeai import GenerativeModel, configure
configure(api_key="YOUR_GEMNI_API_KEY")


In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
from PIL import Image
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from diffusers import StableDiffusionPipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from google.generativeai import GenerativeModel

# Load Stable Diffusion model
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda")

def generate_image(prompt, save_path):
    image = pipe(prompt).images[0]
    image.save(save_path)

# Function to get prompts from Gemini API
def get_prompts_from_gemini(class_names):
    model = GenerativeModel("gemini-pro")
    class_prompts = {}

    for class_name in class_names:
        prompt = f"Generate diverse and detailed image descriptions for the category '{class_name}'."
        response = model.generate_content(prompt)
        class_prompts[class_name] = response.text.split("\n")[:10]  # Extract top prompts

    return class_prompts

def generate_images(classes, num_images=10):
    os.makedirs("dataset_s", exist_ok=True)

    class_prompts = get_prompts_from_gemini(classes)

    for class_name in classes:
        os.makedirs(f"dataset_s/{class_name}", exist_ok=True)

        for i in range(num_images):
            prompt = np.random.choice(class_prompts[class_name])
            save_path = f"dataset_s/{class_name}/{i}.png"
            generate_image(prompt, save_path)

classes = ["studying", "not_studying"]

generate_images(classes, 10)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...

tokenizer%2Ftokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vae%2Fconfig.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Install YOLOv8
!pip install ultralytics -q

# Import necessary libraries
import os
import random
import shutil
import yaml
from ultralytics import YOLO

# Define dataset paths
base_path = "/content/dataset_s"
output_path = "/content/dataset"
image_train_path = os.path.join(output_path, "images/train")
image_val_path = os.path.join(output_path, "images/val")
label_train_path = os.path.join(output_path, "labels/train")
label_val_path = os.path.join(output_path, "labels/val")

# Create required directories
for path in [image_train_path, image_val_path, label_train_path, label_val_path]:
    os.makedirs(path, exist_ok=True)


class_dirs = {cls: idx for idx, cls in enumerate(classes)}




In [ ]:
split_ratio = 0.8

# Function to generate YOLO labels and split dataset
def process_class(class_name, class_id):
    img_folder = os.path.join(base_path, class_name)
    images = [f for f in os.listdir(img_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(images)  # Shuffle for randomness

    train_count = int(len(images) * split_ratio)

    for i, img in enumerate(images):
        img_src = os.path.join(img_folder, img)
        if i < train_count:
            img_dst = os.path.join(image_train_path, img)
            label_folder = label_train_path
        else:
            img_dst = os.path.join(image_val_path, img)
            label_folder = label_val_path

        # Copy image to correct split
        shutil.copy(img_src, img_dst)

        # Create label file
        label_filename = img.rsplit('.', 1)[0] + ".txt"
        label_dst = os.path.join(label_folder, label_filename)

        # YOLO format: class_id x_center y_center width height (full image box)
        with open(label_dst, "w") as f:
            f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")

# Process each class folder
for class_name, class_id in class_dirs.items():
    process_class(class_name, class_id)

print("✅ Dataset split & YOLO labels generated.")


In [ ]:

# Dynamically create dataset.yaml file
names_yaml = "\n".join([f"  {idx}: {name}" for idx, name in enumerate(classes)])
dataset_yaml = f"""
path: /content/dataset
train: images/train
val: images/val

names:
{names_yaml}
"""

yaml_path = "/content/dataset.yaml"
with open(yaml_path, "w") as f:
    f.write(dataset_yaml)

print(f"✅ dataset.yaml created at {yaml_path}")

# Load YOLO model
model = YOLO("yolov8n.pt")  # Use 'yolov8l.pt' for better accuracy

# Train the model
model.train(data=yaml_path, epochs=50, imgsz=640, batch=16, device="cuda")

# Save trained weights
!cp runs/detect/train/weights/best.pt /content/best.pt
print("✅ Model training complete. best.pt saved in /content/")
